In [0]:
df = spark.read.table('dbx_cata.silver.orders')
display(df)

In [0]:
df_cust = spark.sql('SELECT dimCustomerKey, customer_id AS dim_cust_id FROM dbx_cata.gold.customers')

df_prod = spark.sql('SELECT product_id AS dimProductKey, product_id AS dim_prod_id FROM dbx_cata.gold.products')
#used the product_id as surrogate key, since its already unique

In [0]:
spark.read.table('dbx_cata.gold.products').display()

In [0]:
df_fact = df.join(df_cust, df.customer_id == df_cust.dim_cust_id, 'left')\
    .join(df_prod, df.product_id == df_prod.dim_prod_id, 'left')

In [0]:
df_fact.display()

In [0]:
#selecting columns of interest, since the fact table should only contain keys and numerical values
df_fact_final = df_fact.drop('dim_cust_id', 'dim_prod_id', "customer_id", "product_id")

df_fact_final.display()

In [0]:
from delta.tables import DeltaTable

In [0]:
#the primary key/joining column for the orders table is a combination of all the keys in the fact table

if spark.catalog.tableExists('dbx_cata.gold.orders'):
    delta_obj = DeltaTable.forName(spark, 'dbx_cata.gold.orders')
    delta_obj.alias('trg').merge(df_fact_final.alias('src'), 'trg.order_id = src.order_id AND trg.dimCustomerKey = src.dimCustomerKey AND trg.dimProductKey = src.dimProductKey')\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
        
else:
    df_fact_final.write.format('delta').mode('overwrite')\
        .option('path', 'abfss://gold@etedbxcrop.dfs.core.windows.net/orders')\
        .saveAsTable('dbx_cata.gold.orders')

In [0]:
%sql
select * from dbx_cata.gold.orders

In [0]:
%sql
drop table dbx_cata.gold.products

In [0]:
%sql
drop table dbx_cata.gold.products_staging